# 17.5 Homework- Global Optimization I

- write a program to find the ground state of LJ potential for N=3 (assuming $\epsilon= \sigma= 1$)

In order to calculate the LJ potential for N=3 atoms, we shall use the Lennard-Jones Potential equation: 

$$ V= 4\epsilon[ (\frac{\sigma}{r})^2 + (\frac{\sigma}{r})^2)$$

in which $\epsilon$ is the depth of the potential well and $\sigma$ is the distance at which the potential crosses zero, while $r$ is the distance between a pair of atoms. The LJ model consists of two parts: a steep repulsive term and an attractive term, which are expressed by $(\frac{\sigma}{r})^2$ and $(\frac{\sigma}{r})^2$, respectively. 

In [61]:
import numpy as np

def LJ(r): #define Lennard-Jones Potential 
    epsilon=1 #depth of potential well
    sigma=1 #distance at which the potential crosses zero
    r6=r**6 
    r12=r6*r6
    return 4*epsilon*(sigma/r12- sigma/r6)

Since the LJ Potential equation only predicts the potential energy of a pair of atoms, in order to calculate the potential of three atoms, we need to find the distance between all three. We can calculate this by using the following distance equation:

$$ D= \sqrt{(x_{2}- x_{1})^2 - (y_{2}-y_{1})^2}$$

in which the $x$ and $y$ values will be generated randomly. 

After the distances are determined, we can now define a function that would find the lowest energy value after $N$ number of simulations. In theory, the higher the $N$, the more accurate the ground state potential energy would be. 

In [62]:
import time

def distance(x1,x2,y1,y2): #define distance function 
    return np.sqrt(abs((x1-x2)**2 - (y1-y2)**2))

def minimize_LJ(N, L=5): 
    energy_list= [] #store energy values
 
    for i in range(N): #N is the number of simulations 
        Total_energy=0 #starting total energy is zero 
        
        x1= L*np.random.random() #assign random numbers to xs and ys
        x2= L*np.random.random()
        x3= L*np.random.random()

        y1= L*np.random.random()    
        y2= L*np.random.random()
        y3= L*np.random.random()

        r1= distance(x1,x2,y1,y2) #calculate distance between all atoms 
        r2= distance(x1,x3,y1,y3)
        r3= distance(x2,x3,y2,y3)

        Total_energy += LJ(r1) #calculate energy between all atoms 
        Total_energy += LJ(r2)
        Total_energy += LJ(r3)
        
        energy_list.append(Total_energy) 
    
    return min(energy_list) #find the lowest energy value 

start_time=time.time() 
energy=minimize_LJ(1000000)
end_time=time.time()
total_time=end_time-start_time #calculate execution time 
print("The ground state potential is: ", energy)
print ("Time: ",total_time, "seconds")

The ground state potential is:  -2.9979356272513082
Time:  11.053908109664917 seconds


According to the energy values collected by Wales's group in Cambridge, the expected ground potential for 3 atoms is -3.0. Comparing this to our result, we can conclude that our method does work. However, looking at the execution time, this method is not very efficient. In addition, the defined function will be very tedious to use if we want to find the ground state potential of many atoms. In that case, we would have to keep generating new $x$ and $y$ values for every additional atom. 

# 18.3 Homework- Global Optimization II

- Try different minimization methods in scipy on larger systems ($N$ up to 20), and show:
    1. The average number of attempts to find the ground state
    2. The time cost

Instead of using the method from 17.5 homework, we can calculate the number of interatomic pairs using the following:

$$N\times(N-1)/2$$

where $N=3$. For each pair, we would need to calculate the distance $r$, and then evaluate the energy using the LJ potential equation. 

In [63]:
def total_LJ(pos): #calculate total energy of N number of atoms 
    E=0
    N_atom= int(len(pos)/3) 
    
    for i in range(N_atom-1): 
        for j in range(i+1, N_atom):
            Pos1= pos[i*3:(i+1)*3]
            Pos2= pos[j*3:(j+1)*3]
            distance=np.linalg.norm(Pos1-Pos2)
            
            E += LJ(distance)
    return E

Similar to the first method, we would again need to generate random position for $N$ number of atoms. 

In [64]:
def init_pos(N,L=5): #generate random positions for N number of atoms 
    return L*np.random.random_sample((N*3))

We can now use Scipy's minimize function in order to determine the minimum potential energy of $N$ atoms. We will use the following minimization methods and then choose the most efficient one to determine the ground state potential of atoms ranging from 4-20:

    1. BFGS: Broyden-Fletcher-Goldfarb-Shanno method
    2. CG: Conjugate Gradient method
    3. Nelder-Mead method
    4. Powell Method 

In [ ]:
from scipy.optimize import minimize
 
start_time=time.time()
fx_list=[] #store function values 
x_list=[] #store x values 

def ground_state(N_atom, method, N_attempts): #calculate ground state potential of N number of atoms 
    start_time= time.time()
    for i in range(N_attempts):
        pos= init_pos(N_atom)
        results=minimize(total_LJ, pos, method=method, tol=1e-4) #use scipy minimize function to find lowest function value 
        fx_list.append(results.fun)
        x_list.append(results.x)
        if i%10==0: #for every 10 steps, record value at that point and time 
            end_time=time.time()
            total_time= end_time-start_time #calculate execution time 
            print("Method: ", method, 'Step #: ', i, '  Value:', results.fun, "Time: ", total_time, "sec")    
        
    print ("Ground state potential is:", min(fx_list))


In [89]:
ground_state(3, 'BFGS', 50)

Method:  BFGS Step #:  0   Value: -2.9999999999995466 Time:  0.0243222713470459 sec
Method:  BFGS Step #:  10   Value: -2.999999999979061 Time:  0.2830381393432617 sec
Method:  BFGS Step #:  20   Value: -2.999999999994726 Time:  0.5370461940765381 sec
Method:  BFGS Step #:  30   Value: -2.999999999999318 Time:  0.7482199668884277 sec
Method:  BFGS Step #:  40   Value: -2.999999999999146 Time:  1.0107522010803223 sec
Ground state potential is: -2.9999999999999956


In [76]:
ground_state(3, 'CG',50)

Method:  CG Step #:  0   Value: -2.9999999999438223 Time:  0.07292318344116211 sec
Method:  CG Step #:  10   Value: -2.9999999999999987 Time:  0.3857252597808838 sec
Method:  CG Step #:  20   Value: -2.999999999978899 Time:  0.716508150100708 sec
Method:  CG Step #:  30   Value: -2.999999999960896 Time:  0.930408239364624 sec
Method:  CG Step #:  40   Value: -2.999999999985993 Time:  1.2587132453918457 sec
Ground state potential is: -2.999999999999999


In [77]:
ground_state(3, 'Nelder-Mead',50)

Method:  Nelder-Mead Step #:  0   Value: -2.9999999732244422 Time:  0.06808781623840332 sec
Method:  Nelder-Mead Step #:  10   Value: -2.999999998114929 Time:  0.8838379383087158 sec
Method:  Nelder-Mead Step #:  20   Value: -2.9999999982903303 Time:  1.590122938156128 sec
Method:  Nelder-Mead Step #:  30   Value: -2.9999999843935865 Time:  2.1999099254608154 sec
Method:  Nelder-Mead Step #:  40   Value: -2.999999994955434 Time:  2.923142910003662 sec
Ground state potential is: -2.999999999999999


In [78]:
ground_state(3, 'Powell',50 )

Method:  Powell Step #:  0   Value: -2.999999999156739 Time:  0.10570645332336426 sec
Method:  Powell Step #:  10   Value: -2.9999999999997757 Time:  0.7316312789916992 sec
Method:  Powell Step #:  20   Value: -2.999999999982415 Time:  1.1314191818237305 sec
Method:  Powell Step #:  30   Value: -2.999999999076223 Time:  1.5463533401489258 sec
Method:  Powell Step #:  40   Value: -2.99999998282589 Time:  1.9976122379302979 sec
Ground state potential is: -3.0


Looking at the results, all methods calculated accurate results, even with just a few attempts. However, the BFGS method was the fastest. Now, we can calculate the ground state potential energy of $N=4$ to $N=20$, using strictly the BFGS method, with 20 attempts. 

In [90]:
for i in range(4,21):
    print("Number of atoms: ", i)
    print( "Attempts: 20")
    energy= ground_state(i, "BFGS", 20)
    print('\n')
    

Number of atoms:  4
Attempts: 20
Method:  BFGS Step #:  0   Value: -5.999999999932005 Time:  0.09274411201477051 sec
Method:  BFGS Step #:  10   Value: -5.999999999969069 Time:  0.8186490535736084 sec
Ground state potential is: -5.999999999999879


Number of atoms:  5
Attempts: 20
Method:  BFGS Step #:  0   Value: -9.103851818628883 Time:  0.32808971405029297 sec
Method:  BFGS Step #:  10   Value: -9.103852415706752 Time:  2.195434808731079 sec
Ground state potential is: -9.103852415706752


Number of atoms:  6
Attempts: 20
Method:  BFGS Step #:  0   Value: -12.302927529537861 Time:  0.2884559631347656 sec
Method:  BFGS Step #:  10   Value: -12.302927529527082 Time:  3.7113709449768066 sec
Ground state potential is: -12.71206225665728


Number of atoms:  7
Attempts: 20
Method:  BFGS Step #:  0   Value: -15.593210938146438 Time:  0.5279741287231445 sec
Method:  BFGS Step #:  10   Value: -15.533060054496666 Time:  6.063563108444214 sec
Ground state potential is: -16.50538416799992


Numb

Comparing the ground state potentials to the ones collected in Cambrdige, the results are similar. However, from $N$= 18 to $N$=20, the potential energy value starts to deviate from the expected value. Furthermore, the time it takes for the computation to finish increases as the number of atoms increase, which was expected. 